In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.losses import BinaryFocalCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization, Activation
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, Nadam
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
from scipy.spatial.distance import jensenshannon

# Custom metrics with correct casting to avoid float/int issues
def subset_accuracy(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    return tf.reduce_mean(tf.cast(tf.reduce_all(tf.equal(tf.round(y_true), tf.round(y_pred)), axis=1), tf.float32))

def hamming_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    return tf.reduce_mean(tf.cast(tf.not_equal(y_true, tf.round(y_pred)), tf.float32))

def matthews_correlation(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.round(y_pred), tf.float32)

    tp = tf.reduce_sum(tf.cast(y_true * y_pred, tf.float32))
    tn = tf.reduce_sum(tf.cast((1 - y_true) * (1 - y_pred), tf.float32))
    fp = tf.reduce_sum(tf.cast((1 - y_true) * y_pred, tf.float32))
    fn = tf.reduce_sum(tf.cast(y_true * (1 - y_pred), tf.float32))

    numerator = tp * tn - fp * fn
    denominator = tf.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + tf.keras.backend.epsilon())

def macro_f1_score(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.round(y_pred), tf.float32)

    tp = tf.reduce_sum(tf.cast(y_true * y_pred, tf.float32), axis=0)
    fp = tf.reduce_sum(tf.cast((1 - y_true) * y_pred, tf.float32), axis=0)
    fn = tf.reduce_sum(tf.cast(y_true * (1 - y_pred), tf.float32), axis=0)

    precision = tp / (tp + fp + tf.keras.backend.epsilon())
    recall = tp / (tp + fn + tf.keras.backend.epsilon())
    f1 = 2 * precision * recall / (precision + recall + tf.keras.backend.epsilon())

    return tf.reduce_mean(f1)

# Set the best hyperparameters
best_params = {
    'apply_yj': True,
    'apply_scaler': True,
    'batch_size': 781, 
    'n_layers': 3,
    'n_units_0': 875,
    'n_units_1': 938,
    'n_units_2': 402,
    'activation': 'relu',
    'dropout_rate': 0.11698557003292169,
    'apply_batch_norm': True,
    'optimizer': 'adam', # could try 'nadam'
    'regularization': None,
    'reg_lambda': 0.05,
    'learning_rate': 0.00026320814416562444
}
print("Best hyperparameters: ", best_params)

# Load the dataset
df = pd.read_csv('final_dataset.tsv', sep='\t')
feature_cols = ['mH2', 'mHD', 'mAD', 'mHDp', 'alpha', 'L2', 'L8', 'vs', 'm22sq']
label_cols = ['valid_BFB', 'valid_Uni', 'valid_STU', 'valid_Higgs']

X_selected = df[feature_cols].copy()
y = df[label_cols]

# Preprocessing function
def preprocess_data(X, apply_yj=False, apply_scaler=False):
    if apply_yj:
        pt = PowerTransformer(method='yeo-johnson')
        X = pt.fit_transform(X)
    if apply_scaler:
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
    return X

# Split data into training, validation, and test sets (70%, 15%, 15%)
X_temp, X_test, y_temp, y_test = train_test_split(X_selected, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42)

# Apply preprocessing based on the best parameters
X_train_processed = preprocess_data(X_train, best_params['apply_yj'], best_params['apply_scaler'])
X_val_processed = preprocess_data(X_val, best_params['apply_yj'], best_params['apply_scaler'])
X_test_processed = preprocess_data(X_test, best_params['apply_yj'], best_params['apply_scaler'])

# Define the model
def create_model(params, input_shape, num_labels):
    n_layers = params['n_layers']
    units = [params[f'n_units_{i}'] for i in range(n_layers)]
    activation = params['activation']
    dropout_rate = params['dropout_rate']
    apply_batch_norm = params['apply_batch_norm']
    regularization = params['regularization']
    reg_lambda = params['reg_lambda'] if regularization == 'l2' else 0.0
    learning_rate = params['learning_rate']

    model = Sequential()
    model.add(Input(shape=input_shape))

    for unit in units:
        model.add(Dense(unit, kernel_regularizer=l2(reg_lambda) if regularization == 'l2' else None))
        model.add(Activation(activation))
        if apply_batch_norm:
            model.add(BatchNormalization())
        model.add(Dropout(dropout_rate))

    model.add(Dense(num_labels, activation='sigmoid'))

    if params['optimizer'] == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif params['optimizer'] == 'nadam':
        optimizer = Nadam(learning_rate=learning_rate)

    model.compile(
        optimizer=optimizer, 
        loss=BinaryFocalCrossentropy(gamma=2.0, from_logits=False),
        metrics=[subset_accuracy, hamming_loss, matthews_correlation, macro_f1_score]
    )

    return model

# Instantiate and compile the model
model = create_model(best_params, input_shape=(X_train_processed.shape[1],), num_labels=y_train.shape[1])

# Create output directory if it doesn't exist
output_dir = 'Final_results'
os.makedirs(output_dir, exist_ok=True)

# Callbacks for early stopping and model checkpointing
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint_path = os.path.join(output_dir, 'best_model.keras')
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(
    X_train_processed, y_train,
    validation_data=(X_val_processed, y_val),
    batch_size=best_params['batch_size'],
    epochs=70,
    verbose=1,
    callbacks=[early_stopping, checkpoint]
)

# Save the full model and training history
model.save(os.path.join(output_dir, 'final_model.keras'))

# Save training history as JSON
history_dict = history.history
with open(os.path.join(output_dir, 'training_history.json'), 'w') as f:
    json.dump(history_dict, f)

# Evaluate on the test set
test_metrics = model.evaluate(X_test_processed, y_test)
y_pred_proba = model.predict(X_test_processed)
y_pred_classes = (y_pred_proba > 0.5).astype(int)

# Print Subset Accuracy on Test Set
test_subset_accuracy = subset_accuracy(y_test, y_pred_classes).numpy()
print(f"Subset Accuracy on Test Set: {test_subset_accuracy:.4f}")

# Print and save classification reports for each label
y_combined_test = y_test.to_numpy()

for i, label in enumerate(label_cols):
    print(f"Classification report for {label}:")
    report = classification_report(y_combined_test[:, i], y_pred_classes[:, i])
    print(report)
    
    with open(os.path.join(output_dir, f'classification_report_{label}.txt'), 'w') as f:
        f.write(f"Classification report for {label}:\n")
        f.write(report)
    
    print(f"\nClassification report for {label} (focusing on class 0):")
    report_class0 = classification_report(y_combined_test[:, i], y_pred_classes[:, i], labels=[0], target_names=['class_0'], zero_division=1)
    print(report_class0)
    
    with open(os.path.join(output_dir, f'classification_report_{label}_class0.txt'), 'w') as f:
        f.write(f"Classification report for {label} (focusing on class 0):\n")
        f.write(report_class0)

# Plot confusion matrices for each label
descriptive_titles = {
    'valid_BFB': 'BfB',
    'valid_Uni': 'PU',
    'valid_STU': 'STU',
    'valid_Higgs': 'Higgs'
}

for i, label in enumerate(label_cols):
    cm = confusion_matrix(y_combined_test[:, i], y_pred_classes[:, i])
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.title(f'Confusion Matrix for {descriptive_titles[label]}\nin Combined Test Set', fontsize=16)
    plt.xlabel('Predicted', fontsize=14)
    plt.ylabel('Actual', fontsize=14)
    plt.xticks(np.arange(2) + 0.5, ['0', '1'], rotation=0, fontsize=12)
    plt.yticks(np.arange(2) + 0.5, ['0', '1'], rotation=0, fontsize=12)
    
    for (j, k), value in np.ndenumerate(cm):
        plt.text(k + 0.5, j + 0.5, f'{value}', ha='center', va='center',
                 bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='white'),
                 fontsize=14, weight='bold')

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'confusion_matrix_{label}_combined.png'))
    plt.close()

# Plot training and validation metrics
metrics_pairs = [
    ('subset_accuracy', 'val_subset_accuracy'),
    ('hamming_loss', 'val_hamming_loss'),
    ('matthews_correlation', 'val_matthews_correlation'),
    ('macro_f1_score', 'val_macro_f1_score'),
    ('loss', 'val_loss')
]

for train_metric, val_metric in metrics_pairs:
    plt.figure()
    plt.plot(history.history[train_metric], label=f'Training {train_metric}')
    plt.plot(history.history[val_metric], label=f'Validation {val_metric}')
    plt.title(f'{train_metric} vs. {val_metric}')
    plt.xlabel('Epochs')
    plt.ylabel(train_metric)
    plt.legend(loc='best')
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, f'{train_metric}_vs_{val_metric}_history.png'), dpi=300, bbox_inches='tight')
    plt.show()

# Analysis on specific combinations of labels
unique_combinations = np.unique(y_combined_test, axis=0)
for combination in unique_combinations:
    mask = np.all(y_combined_test == combination, axis=1)
    subset_true = y_combined_test[mask]
    subset_pred = y_pred_classes[mask]
    
    print(f"\nPerformance for combination {combination}:")
    report_combination = classification_report(subset_true, subset_pred, target_names=label_cols)
    print(report_combination)
    
    with open(os.path.join(output_dir, f'classification_report_combination_{"_".join(map(str, combination))}.txt'), 'w') as f:
        f.write(f"Performance for combination {combination}:\n")
        f.write(report_combination)
    
    # Save misclassified instances for each combination
    misclassified_mask = ~np.all(subset_true == subset_pred, axis=1)
    misclassified_indices = np.where(mask)[0][misclassified_mask]
    misclassified_data = pd.DataFrame(X_test_processed[misclassified_indices], columns=feature_cols)
    misclassified_data[label_cols] = y_combined_test[misclassified_indices]
    misclassified_data['predicted_' + '_'.join(label_cols)] = [tuple(row) for row in y_pred_classes[misclassified_indices]]
    misclassified_data.to_csv(os.path.join(output_dir, f'misclassified_combination_{"_".join(map(str, combination))}.csv'), index=False)
    print(f"Misclassified data for combination {combination} saved.")

print("All evaluation outputs have been saved to the output directory.")


###################################
# ADDITIONAL EVALUATIONS
###################################

# 1. Powerset-based evaluation:
# Treat each unique combination as a class and measure macro F1 across these combination-classes.
print("\n=== Powerset-Based Evaluation ===")
# Map each combination to an integer class
combo_to_class = {tuple(c): idx for idx, c in enumerate(unique_combinations)}

y_true_combos = [combo_to_class[tuple(row)] for row in y_combined_test]
y_pred_combos = [combo_to_class[tuple(row)] for row in y_pred_classes]

powerset_precision = precision_score(y_true_combos, y_pred_combos, average='macro', zero_division=1)
powerset_recall = recall_score(y_true_combos, y_pred_combos, average='macro', zero_division=1)
powerset_f1 = f1_score(y_true_combos, y_pred_combos, average='macro', zero_division=1)

print(f"Powerset Macro-F1: {powerset_f1:.4f}")
print(f"Powerset Macro-Precision: {powerset_precision:.4f}")
print(f"Powerset Macro-Recall: {powerset_recall:.4f}")

powerset_report = classification_report(y_true_combos, y_pred_combos, zero_division=1)
print("Powerset Classification Report:\n", powerset_report)
with open(os.path.join(output_dir, 'powerset_classification_report.txt'), 'w') as f:
    f.write("Powerset Classification Report:\n")
    f.write(powerset_report)


# 2. Distributional comparisons (Jensen-Shannon Divergence):
print("\n=== Distributional Comparison ===")
# Compute empirical distribution of true combinations
unique_combo_indices, true_counts = np.unique(y_true_combos, return_counts=True)
unique_combo_indices_pred, pred_counts = np.unique(y_pred_combos, return_counts=True)

# Create full distributions (some combos might not appear in predictions or vice versa)
num_combos = len(unique_combinations)
true_dist = np.zeros(num_combos, dtype=np.float32)
pred_dist = np.zeros(num_combos, dtype=np.float32)

for idx, count in zip(unique_combo_indices, true_counts):
    true_dist[idx] = count
for idx, count in zip(unique_combo_indices_pred, pred_counts):
    pred_dist[idx] = count

true_dist /= np.sum(true_dist)
pred_dist /= np.sum(pred_dist)

# JS divergence between distributions (JS is symmetric and sqrt(JS) is a metric)
js_div = jensenshannon(true_dist, pred_dist)**2
print(f"Jensen-Shannon Divergence between true and predicted combination distributions: {js_div:.6f}")
with open(os.path.join(output_dir, 'distribution_comparison.txt'), 'w') as f:
    f.write(f"Jensen-Shannon Divergence: {js_div}\n")


# 3. Conditional Probability Checks:
print("\n=== Conditional Probability Checks ===")
# For each pair of labels i, j, check P(y_j=1 | y_i=1) in true and predicted sets
def conditional_probabilities(y_true, y_pred):
    # y_true and y_pred are binary arrays of shape (N, L)
    L = y_true.shape[1]
    results = []
    for i in range(L):
        for j in range(L):
            if i == j:
                continue
            mask_i = (y_true[:, i] == 1)
            true_cond = np.mean(y_true[mask_i, j]) if np.sum(mask_i) > 0 else 0.0

            mask_i_pred = (y_pred[:, i] == 1)
            pred_cond = np.mean(y_pred[mask_i_pred, j]) if np.sum(mask_i_pred) > 0 else 0.0

            results.append((label_cols[i], label_cols[j], true_cond, pred_cond))
    return results

cond_probs = conditional_probabilities(y_combined_test, y_pred_classes)
for (li, lj, t_c, p_c) in cond_probs:
    print(f"P({lj}=1|{li}=1) True: {t_c:.3f}, Pred: {p_c:.3f}")

with open(os.path.join(output_dir, 'conditional_probabilities.txt'), 'w') as f:
    f.write("Conditional Probabilities (P(label_j=1|label_i=1))\n")
    for (li, lj, t_c, p_c) in cond_probs:
        f.write(f"{li}->{lj}: True={t_c:.3f}, Pred={p_c:.3f}\n")

print("\nAdditional evaluation outputs have been saved to the output directory.")
